<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab 3.02: Statistical Modeling and Model Validation

> Authors: Tim Book, Matt Brems

---

## Objective
The goal of this lab is to guide you through the modeling workflow. In this lesson, you will follow all best practices when slicing your data and validating your model. The goal of this lab is not necessarily to build the best model you can, but to build and evaluate a model, and interpret its results.

## Imports

In [1]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
# Muscle memory is important!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [2]:
# Read in the citibike data in the data folder in this repository.
citibike = pd.read_csv('./data/citibike_feb2014.csv')

## Explore the data
Use this space to familiarize yourself with the data.

Convince yourself there are no issues with the data. If you find any issues, clean them here.

In [3]:
citibike.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,21101,Subscriber,1991,1


In [4]:
# Birth year should be an integer
citibike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224736 entries, 0 to 224735
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             224736 non-null  int64  
 1   starttime                224736 non-null  object 
 2   stoptime                 224736 non-null  object 
 3   start station id         224736 non-null  int64  
 4   start station name       224736 non-null  object 
 5   start station latitude   224736 non-null  float64
 6   start station longitude  224736 non-null  float64
 7   end station id           224736 non-null  int64  
 8   end station name         224736 non-null  object 
 9   end station latitude     224736 non-null  float64
 10  end station longitude    224736 non-null  float64
 11  bikeid                   224736 non-null  int64  
 12  usertype                 224736 non-null  object 
 13  birth year               224736 non-null  object 
 14  gend

In [5]:
# How to change the 
citibike['birth year'].unique()

array(['1991', '1979', '1948', '1981', '1990', '1978', '1944', '1983',
       '1969', '1986', '1962', '1965', '1942', '1989', '1980', '1957',
       '1951', '1992', '1971', '1982', '1968', '1984', '\\N', '1956',
       '1987', '1985', '1996', '1975', '1988', '1974', '1972', '1959',
       '1973', '1977', '1976', '1953', '1993', '1970', '1963', '1967',
       '1966', '1960', '1961', '1994', '1958', '1955', '1946', '1964',
       '1900', '1995', '1954', '1952', '1949', '1947', '1941', '1938',
       '1950', '1945', '1997', '1934', '1940', '1939', '1936', '1943',
       '1935', '1937', '1922', '1932', '1907', '1926', '1899', '1901',
       '1917', '1910', '1933', '1921', '1927', '1913'], dtype=object)

In [7]:
def clean_by(cell):
    if cell == '\\N':
        return 1979
    else:
        return float(cell)

In [8]:
citibike['birth year'] = citibike['birth year'].map(clean_by)

In [9]:
citibike.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                 float64
gender                       int64
dtype: object

In [10]:
# There are no null values
citibike.isna().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

### Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

In [11]:
citibike['gender'].value_counts()

1    176526
2     41479
0      6731
Name: gender, dtype: int64

**Null Hypothesis:** On average the trip duration is not different by gender.  
**Alternative Hypothesis:** On average the trip duration is different by gender. 

In [12]:
from scipy import stats

In [13]:
trt = citibike[citibike['gender'] == 1]['tripduration']
ctrl = citibike[citibike['gender'] == 2]['tripduration']

In [14]:
# Conduct our t-test.
tt = stats.ttest_ind(trt, ctrl, equal_var=False)
tt

Ttest_indResult(statistic=-4.802922158264667, pvalue=1.5680482053980446e-06)

In [15]:
# Print the average of the control and experimental groups.
print(trt.mean())
print(ctrl.mean())

814.0324088236293
991.3610742785506


In [16]:
trt.mean() - ctrl.mean()

-177.3286654549213

In [17]:
tt.pvalue

1.5680482053980446e-06

**Answer:** 
Since our p-value is below our significance level of 0.05, we succeed in rejecting the null hypothesis.

### What numeric columns shouldn't be treated as numeric?

**Answer:** The columns 'start station id' should not  be treated as a numeric column.

### Dummify the `start station id` Variable

In [18]:
citibike['start station id'].value_counts()

293     2920
519     2719
497     2493
435     2403
521     2171
        ... 
431       54
278       45
443       41
2005      36
320        4
Name: start station id, Length: 329, dtype: int64

In [19]:
df = pd.get_dummies(citibike['start station id'], drop_first=True)

In [20]:
citibike = citibike.join(df)

In [21]:
citibike

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,2006,2008,2009,2010,2012,2017,2021,2022,2023,3002
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,...,0,0,0,0,0,0,0,0,0,0
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,...,0,0,0,0,0,0,0,0,0,0
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,...,0,0,0,0,0,0,0,0,0,0
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,...,0,0,0,0,0,0,0,0,0,0
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224731,848,2014-02-28 23:57:13,2014-03-01 00:11:21,498,Broadway & W 32 St,40.748549,-73.988084,432,E 7 St & Avenue A,40.726218,...,0,0,0,0,0,0,0,0,0,0
224732,1355,2014-02-28 23:57:55,2014-03-01 00:20:30,470,W 20 St & 8 Ave,40.743453,-74.000040,302,Avenue D & E 3 St,40.720828,...,0,0,0,0,0,0,0,0,0,0
224733,304,2014-02-28 23:58:17,2014-03-01 00:03:21,497,E 17 St & Broadway,40.737050,-73.990093,334,W 20 St & 7 Ave,40.742388,...,0,0,0,0,0,0,0,0,0,0
224734,308,2014-02-28 23:59:10,2014-03-01 00:04:18,353,S Portland Ave & Hanson Pl,40.685396,-73.974315,365,Fulton St & Grand Ave,40.682232,...,0,0,0,0,0,0,0,0,0,0


In [22]:
citibike.isna().sum()

tripduration          0
starttime             0
stoptime              0
start station id      0
start station name    0
                     ..
2017                  0
2021                  0
2022                  0
2023                  0
3002                  0
Length: 343, dtype: int64

In [23]:
citibike['birth year']

0         1991.0
1         1979.0
2         1948.0
3         1981.0
4         1990.0
           ...  
224731    1976.0
224732    1985.0
224733    1968.0
224734    1982.0
224735    1960.0
Name: birth year, Length: 224736, dtype: float64

## Feature Engineering
Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected)
- Note: you will need to clean the data a bit.

In [24]:
citibike.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,2006,2008,2009,2010,2012,2017,2021,2022,2023,3002
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,...,0,0,0,0,0,0,0,0,0,0


In [25]:
citibike['age'] = 2014 - citibike['birth year']

## Split your data into train/test sets

Look at the size of your data. What is a good proportion for your split? **Justify your answer, considering the size of your data and the default split size in sklearn.**

Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `usertype`, `gender`, and the dummy variables you created from `start station id`. (Hint: You may find the Pandas `.drop()` method helpful here.) 

In [26]:
citibike.columns

Index([           'tripduration',               'starttime',
                      'stoptime',        'start station id',
            'start station name',  'start station latitude',
       'start station longitude',          'end station id',
              'end station name',    'end station latitude',
       ...
                            2008,                      2009,
                            2010,                      2012,
                            2017,                      2021,
                            2022,                      2023,
                            3002,                     'age'],
      dtype='object', length=344)

In [27]:
dropped_df = citibike.drop(columns = ['tripduration', 'stoptime', 'start station name', 'start station longitude', 'end station name',
                        'starttime', 'start station id', 'start station latitude', 'end station id', 'end station id',
                                     'end station longitude', 'end station latitude', 'birth year'])

In [28]:
dropped_df.head(1)

,bikeid,usertype,gender,79,82,83,116,119,120,127,...,2008,2009,2010,2012,2017,2021,2022,2023,3002,age
0,21101,Subscriber,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,23.0


In [29]:
user_dum = pd.get_dummies(dropped_df['usertype'], drop_first=True)

In [30]:
user_dum

,Subscriber
0,1
1,1
2,1
3,1
4,1
...,...
224731,1
224732,1
224733,1
224734,1


In [31]:
dropped_df = dropped_df.join(user_dum)

In [32]:
dropped_df.head(1)

,bikeid,usertype,gender,79,82,83,116,119,120,127,...,2009,2010,2012,2017,2021,2022,2023,3002,age,Subscriber
0,21101,Subscriber,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23.0,1


In [48]:
X = dropped_df.drop(columns=['bikeid', 'usertype'])

In [49]:
X.head(1)

,gender,79,82,83,116,119,120,127,128,137,...,2009,2010,2012,2017,2021,2022,2023,3002,age,Subscriber
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23.0,1


In [50]:
y = citibike['tripduration']

## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size = .65)

In [52]:
model = LinearRegression()

In [53]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [54]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [55]:
y_pred_test.mean()

876.5389452686321

In [56]:
model.score(X_train, y_train)

0.0054176987603250515

In [57]:
model.score(X_test, y_test)

-0.0031713904716592634

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ to be the mean of our training `y` values.)

The model was underfit. The 

In [58]:
y_null = np.zeros_like(y_train)
y_null = y_null + citibike['tripduration'].mean()
y_null[0:5]

array([874.51980991, 874.51980991, 874.51980991, 874.51980991,
       874.51980991])

In [59]:
mean_squared_error(y_train, y_pred_train)

31803606.675827023

In [60]:
mean_squared_error(y_train, y_null, squared=False)

5654.807480337054

## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

In [64]:
import statsmodels.api as sm

In [66]:
mod = sm.OLS(y_train, X_train).fit()

In [67]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           tripduration   R-squared (uncentered):                   0.028
Model:                            OLS   Adj. R-squared (uncentered):              0.026
Method:                 Least Squares   F-statistic:                              12.87
Date:                Sun, 09 Aug 2020   Prob (F-statistic):                        0.00
Time:                        21:21:48   Log-Likelihood:                     -1.4690e+06
No. Observations:              146078   AIC:                                  2.939e+06
Df Residuals:                  145747   BIC:                                  2.942e+06
Df Model:                         331                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
gender       180.0216     38.551      4.670      0.000     104.463     255.581
79          1688.7289    291.411      5.795      0.000    1117.569    2259.889
82          2037.8557    410.172      4.968      0.000    1233.926    2841.786
83          1358.6681    422.466      3.216      0.001     530.644    2186.692
116         1089.2042    212.924      5.115      0.000     671.877    1506.532
119         1228.9840    958.470      1.282      0.200    -649.598    3107.566
120         1228.0125    728.919      1.685      0.092    -200.655    2656.680
127         1214.2890    238.608      5.089      0.000     746.622    1681.956
128         1211.5096    221.705      5.465      0.000     776.971    1646.048
137         1247.6230    326.852      3.817      0.000     607.000    1888.246
143         1199.1204    525.872      2.280      0.023     168.422    2229.819
144         1407.4494    795.618      1.769      0.077    -151.946    2966.845
146         1044.8581    320.649      3.259      0.001     416.392    1673.325
147         1458.1033    243.809      5.981      0.000     980.242    1935.964
150         1134.2439    311.233      3.644      0.000     524.233    1744.254
151         1131.9571    186.550      6.068      0.000     766.323    1497.591
152         1167.7653    367.260      3.180      0.001     447.943    1887.588
153         1103.2876    236.815      4.659      0.000     639.136    1567.440
157         1787.9666    421.137      4.246      0.000     962.547    2613.386
160         1417.2406    290.847      4.873      0.000     847.186    1987.295
161         1148.4031    218.344      5.260      0.000     720.454    1576.353
164         1318.3651    286.167      4.607      0.000     757.483    1879.247
167         1324.4526    243.839      5.432      0.000     846.533    1802.372
168         1269.7990    213.543      5.946      0.000     851.258    1688.340
173         1203.0610    244.252      4.925      0.000     724.332    1681.790
174         1276.9394    263.194      4.852      0.000     761.084    1792.795
195         2023.7785    284.166      7.122      0.000    1466.819    2580.738
212         1269.3493    237.627      5.342      0.000     803.606    1735.093
216         1353.2825    829.156      1.632      0.103    -271.846    2978.411
217         1449.0470    586.302      2.472      0.013     299.907    2598.187
218         1946.3651    828.617      2.349      0.019     322.293    3570.437
223         1409.0170    250.086      5.634      0.000     918.853    1899.181
224         1189.5476    322.246      3.691      0.000     557.952    1821.143
225         1322.7691    381.295      3.469      0.001     575.438    2070.100
228         1424.2142    287.871      4.947      0.000     85

## Evaluate your model
Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

With this specific model age does not have a significant effect when predicting trip duration. Since we did 'get dummies' on the station id there are to many columns for the model which resulted in high variance. Since our p-value is below our significance level of 0.05, we succeed in rejecting the null hypothesis proving that on average the trip duration is different by gender.

## Citi Bike is attempting to market to people who they think will ride their bike for a long time. Based on your modeling, what types of individuals should Citi Bike market toward?

Based on our model it is hard to predict what types of individuals would ride their bikes for the longest. However, when looking at the raw numbers gender 2 has a higher average trip duration. 